In [ ]:
!pip install PyPDF2
!pip install pdfplumber


#import necessary libraries 

from PyPDF2 import PdfReader
import pdfplumber
import re 
import hashlib
import json 
import os

#define the location of the pdf files

pdf_files_loc = r"C:\Users\chira\Desktop\Origin\data-operations_role-challenge-1_PDF-anonymization-extraction-20240808T052806Z-001 (2)\data-operations_role-challenge-1_PDF-anonymization-extraction\assets\PDFs"
file_names = os.listdir(pdf_files_loc)


#### EXTRACT

# extract data with pdfplumber - better results than PdfReader
def extract_pdf_data(pdf_files_loc):
    with pdfplumber.open(pdf_files_loc) as pdf:
        for page in pdf.pages:
            text =  page.extract_text()
    return text

#### TRANSFORM

def transform_data(text):

    #patient details
    patient_id = re.search(r"Patient\s*ID\s*:\s*(\d+)", text).group(1)
    patient_age = re.search(r"Age\s*:\s*((\d+)\s*years)", text).group(1)
    gestational_age = re.search(r"GA\s*:\s*((\d+)\s*weeks\s*(\d+)\s*day)", text).group(1)
    bmi = re.search(r"BMI\s*:\s*(\d+)", text).group(1)

    #examination findings 
    head_info = re.search(r"Head\s*:\s*(.*)", text).group(1)
    brain_info = re.search(r"Brain\s*:\s*(.*)", text).group(1)
    heart_info = re.search(r"(.*)\nHeart\s*:", text).group(1)
    spine_info = re.search(r"Spine\s*:\s*(.*)", text).group(1)
    abdominal_wall_info = re.search(r"Abdominal\s*wall\s*:\s*(.*)", text).group(1)
    urinary_tract_info = re.search(r"Urinary\s*tract\s*:\s*(.*)", text).group(1)
    extremities_info = re.search(r"Extremities\s*:\s*(.*)", text).group(1)


    examination_findings = dict(zip(['head' , 'brain' , 'heart' ,'spine' , 'abdominal_wall' , 'urinary_tract' , 'extremities'], 
                                    [head_info , brain_info , heart_info , spine_info , abdominal_wall_info , urinary_tract_info , extremities_info]))
    
    #anonymization of patient ID - hashing because it is difficult to decode
    anonymized_patient_id = hashlib.md5(patient_id.encode()).hexdigest()

    patient_doc = {
        'patient_id' : anonymized_patient_id,
        'gestaional_age' : gestational_age, 
        'demographic_age' : patient_age ,
        'BMI' : bmi , 
        'examination_findings' : examination_findings
    }


    return patient_doc, anonymized_patient_id



output_file_path = "C:\\Users\\chira\\Desktop\\Origin\\task1_output\\"

def store_data(id, data):
    write_to_file(id, data)
    print("Data has been written to file")

# function to write the transformed data to a json file
def write_to_file(id,data):
    file_name=output_file_path+id+".json"
    with open(file_name, 'w') as json_file:
        json.dump(data, json_file, indent=4)


def main():
		for file in file_names:
				text = extract_pdf_data(pdf_files_loc+"\\"+file)
				patient_doc, patient_id = transform_data(text)
				store_data(patient_id,patient_doc)

main()

# transformed_data = transform_data(text)
# write_to_file("",transformed_data)
# patient_info_json = json.dumps(transformed_data , indent=4) #converting into JSON format

# print(patient_info_json)